In [ ]:
import os
import sys

if "google.colab" in sys.modules:
    # Running in Colab

    !git clone https://github.com/pthengtr/kcw-analytics.git
    !cd /content/kcw-analytics && git pull origin main

    from google.colab import drive
    drive.mount("/content/drive")

    BASE_FOLDER = "/content/drive/Shareddrives"
    BASE_FOLDER_GIT = "/content"
else:
    # Running in local Jupyter
    BASE_FOLDER = r"G:\Shared drives"
    BASE_FOLDER_GIT = r"C:\Users\Windows 11\Notebook"

print("Using folder:", BASE_FOLDER)

fatal: destination path 'kcw-analytics' already exists and is not an empty directory.
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 7.08 KiB | 2.36 MiB/s, done.
From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
   a390f23..ccb8037  main       -> origin/main
Updating a390f23..ccb8037
Fast-forward
 notebooks/30_vat_sales_report_excel.ipynb         | 130 +++---
 notebooks/31_vat_purchase_report_excel copy.ipynb | 528 ++++------------------
 2 files changed, 148 insertions(+), 510 deletions(-)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using folder: /content/drive/Shareddrives


In [ ]:
from datetime import datetime
import pytz

# Singapore timezone
tz = pytz.timezone("Asia/Singapore")

today = datetime.now(tz)

YEAR = today.year
MONTH = today.month

print(YEAR, MONTH)

In [ ]:
import os
import pandas as pd

def load_csv_folder(folder):
    """
    Load all CSV files from a folder into a dictionary of DataFrames.

    Parameters
    ----------
    folder : str or Path
        Folder path containing CSV files.

    Returns
    -------
    dict[str, pd.DataFrame]
        Dictionary where key = filename, value = DataFrame.
    """

    data = {}

    folder = str(folder)  # ensures Path() also works

    for file in os.listdir(folder):
        if file.lower().endswith(".csv"):
            path = os.path.join(folder, file)

            df = pd.read_csv(
                path,
                dtype={
                    "BCODE": "string",
                    "MOBILE": "string",
                    "BILLNO": "string",
                },
                encoding="utf-8-sig",
                low_memory=False
            )

            data[file] = df
            print(f"Loaded: {file} -> {df.shape}")

    return data

In [ ]:
folder = os.path.join(
    BASE_FOLDER,
    "KCW-Data",
    "kcw_analytics",
    "02_staging",
    "VAT_Sales",
    f"{YEAR}_{MONTH:02d}"
)

data_vat_sales = load_csv_folder(folder)

Loaded: TD.csv -> (149, 10)
Loaded: TAD.csv -> (571, 10)
Loaded: TR.csv -> (32, 10)
Loaded: 3TD.csv -> (0, 38)
Loaded: CN.csv -> (35, 10)
Loaded: 3TAD.csv -> (0, 38)
Loaded: 3TR.csv -> (9, 10)
Loaded: 3CN.csv -> (0, 38)


In [ ]:
folder = os.path.join(
    BASE_FOLDER,
    "KCW-Data",
    "kcw_analytics",
    "04_outputs",
    "TAR",
    f"TAR_{YEAR}_{MONTH:02d}",
    "CSV"
)

data_tar = load_csv_folder(folder)

Loaded: CNTAR_2026_2_summary.csv -> (10, 9)
Loaded: TAR_2026_2_summary.csv -> (225, 8)


In [ ]:
folder = os.path.join(
    BASE_FOLDER,
    "KCW-Data",
    "kcw_analytics",
    "04_outputs",
    "3TAR",
    f"3TAR_{YEAR}_{MONTH:02d}",
    "CSV"
)

data_3tar = load_csv_folder(folder)

Loaded: 3TAR_2026_2_summary.csv -> (118, 8)
Loaded: 3CNTAR_2026_2_summary.csv -> (0, 9)


In [ ]:
df_tar = data_tar[f"TAR_{YEAR}_{MONTH}_summary.csv"].copy()
df_cntar = data_tar[f"CNTAR_{YEAR}_{MONTH}_summary.csv"].copy()

df_3tar = data_3tar[f"3TAR_{YEAR}_{MONTH}_summary.csv"].copy()
df_3cntar = data_3tar[f"3CNTAR_{YEAR}_{MONTH}_summary.csv"].copy()

df_td = data_vat_sales["TD.csv"].copy()
df_tad = data_vat_sales["TAD.csv"].copy()
df_tr = data_vat_sales["TR.csv"].copy()
df_cn = data_vat_sales["CN.csv"].copy()

df_3td = data_vat_sales["3TD.csv"].copy()
df_3tad = data_vat_sales["3TAD.csv"].copy()
df_3tr = data_vat_sales["3TR.csv"].copy()
df_3cn = data_vat_sales["3CN.csv"].copy()

In [ ]:
mask = df_3cn["BILLNO"].str.startswith("3CNTAD", na=False)

# rows that match
df_3cntad = df_3cn[mask].copy()

# remaining rows
df_3cn = df_3cn[~mask].copy()

In [ ]:
mask = df_cn["BILLNO"].str.startswith("CNTAD", na=False)

# rows that match
df_cntad = df_cn[mask].copy()

# remaining rows
df_cn = df_cn[~mask].copy()

In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter

def export_vat_report(
    df,
    sheet_name,
    title_name,
    wb=None,
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สำนักงานใหญ่)",
    company_address="305 หมู่ 1 ต.ชุมแสง อ.วังจันทร์ จ.ระยอง 21210",
    tax_id="0215560000262",
    overwrite_sheet=True,   # if sheet exists, replace it
):
    # ✅ Safe df check (NO "if df:" allowed)
    if df is None:
        raise ValueError("df is None")
    if isinstance(df, pd.Series):
        raise ValueError("df must be a DataFrame, not a Series")
    if df.empty:
        # still create sheet with headers/title, or just skip
        pass

    # Workbook handling
    if wb is None:
        wb = Workbook()
        # remove default sheet if empty default
        if wb.sheetnames == ["Sheet"]:
            wb.remove(wb["Sheet"])

    # Handle existing sheet name
    if sheet_name in wb.sheetnames:
        if overwrite_sheet:
            wb.remove(wb[sheet_name])
        else:
            # make a unique name
            i = 2
            base = sheet_name
            while f"{base}_{i}" in wb.sheetnames:
                i += 1
            sheet_name = f"{base}_{i}"

    ws = wb.create_sheet(sheet_name)

    # Styles
    bold = Font(bold=True)
    center = Alignment(horizontal="center", vertical="center", wrap_text=True)
    left = Alignment(horizontal="left", vertical="center", wrap_text=True)
    header_fill = PatternFill(start_color="F3DFD2", fill_type="solid")
    thin = Side(style="thin")
    border = Border(left=thin, right=thin, top=thin, bottom=thin)

    # Title / Header area
    ws.merge_cells("A1:H1")
    ws["A1"] = title_name
    ws["A1"].font = Font(bold=True, size=14)
    ws["A1"].alignment = center

    ws.merge_cells("A3:C3"); ws["A3"] = "ชื่อสถานประกอบกิจการ"
    ws.merge_cells("D3:F3"); ws["D3"] = company_name
    ws.merge_cells("A4:C4"); ws["A4"] = "ที่อยู่สถานประกอบกิจการ"
    ws.merge_cells("D4:F4"); ws["D4"] = company_address
    ws.merge_cells("A5:C5"); ws["A5"] = "เลขประจำตัวผู้เสียภาษี"
    ws.merge_cells("D5:F5"); ws["D5"] = tax_id

    for r in [3, 4, 5]:
        ws[f"A{r}"].font = bold
        ws[f"A{r}"].alignment = left
        ws[f"D{r}"].alignment = left

    # Table start
    start_row = 7

    # Write df (header + rows)
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start_row):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            cell.border = border
            if r_idx == start_row:
                cell.font = bold
                cell.fill = header_fill
                cell.alignment = center
            else:
                cell.alignment = left

    ws.freeze_panes = f"A{start_row+1}"

    # Column widths (safe)
    for c_idx, col_name in enumerate(df.columns, 1):
        series = df[col_name].astype(str).fillna("")
        max_len = max(series.map(len).max(), len(str(col_name)))
        ws.column_dimensions[get_column_letter(c_idx)].width = min(max_len + 2, 60)

    # Money formatting
    money_like = {"มูลค่าสินค้า", "ภาษีมูลค่าเพิ่ม", "ยอดสุทธิ", "AMOUNT", "VAT", "TOTAL"}
    for c_idx, col_name in enumerate(df.columns, 1):
        name = str(col_name).strip()
        if name in money_like or any(k in name.upper() for k in ["AMOUNT", "VAT", "TOTAL"]):
            for r in range(start_row + 1, start_row + len(df) + 1):
                ws.cell(row=r, column=c_idx).number_format = '#,##0.00'

    date_cols = {"วันที่", "BILLDATE"}
    for c_idx, col_name in enumerate(df.columns, 1):
        if str(col_name).strip() in date_cols:
            for r in range(start_row + 1, start_row + len(df) + 1):
                cell = ws.cell(row=r, column=c_idx)
                cell.number_format = "dd/mm/yyyy"   # <-- your short format

    return wb

In [ ]:
def clean_excel_text(df):
    return df.apply(
        lambda col: col.str.replace(r'^="|"$', '', regex=True)
        if col.dtype == "object" or str(col.dtype) == "string"
        else col
    )

In [ ]:
import pandas as pd

def build_vat_report(
    df,
    rename_map=None,
    column_order=None,
    wb=None,
    sheet_name="REPORT",
    title_name="รายงานภาษี",
    clean_text=True,
    reset_seq=True,
    seq_col="SEQ",
    seq_out_col="ลำดับ",
    add_total_row=True,
    total_sum_cols=("มูลค่าสินค้า", "ภาษีมูลค่าเพิ่ม", "ยอดสุทธิ"),
    total_label_col="เลขที่",
    total_label_text="รวม",
    empty_action="write_headers",   # "write_headers" | "skip"
    **export_kwargs
):
    if df is None:
        raise ValueError("df is None")
    if isinstance(df, pd.Series):
        raise ValueError("df must be a DataFrame, not a Series")

    df = df.copy()

    # optional clean excel ="..."
    if clean_text:
        df = clean_excel_text(df)

    # reset seq (supports either SEQ or ลำดับ)
    if reset_seq:
        df = df.reset_index(drop=True)
        if seq_col in df.columns:
            df[seq_col] = range(1, len(df) + 1)
        elif seq_out_col in df.columns:
            df[seq_out_col] = range(1, len(df) + 1)
        else:
            # create SEQ by default if neither exists
            df[seq_out_col] = range(1, len(df) + 1)

    # rename ONLY if rename_map is given AND it seems applicable
    if rename_map:
        # apply rename only if at least one key exists in df
        if any(k in df.columns for k in rename_map.keys()):
            df = df.rename(columns=rename_map)

    # ✅ EMPTY DF HANDLING (place this BEFORE column_order validation)
    if df.empty:
        if empty_action == "skip":
            return df, wb  # do nothing
        # else: write empty sheet with headers if possible
        if column_order:
            df = pd.DataFrame(columns=list(column_order))
        # export empty header sheet
        wb = export_vat_report(
            df,
            sheet_name=sheet_name,
            title_name=title_name,
            wb=wb,
            **export_kwargs
        )
        return df, wb

    # reorder if provided
    if column_order:
        missing = [c for c in column_order if c not in df.columns]
        if missing:
            raise ValueError(f"Missing columns for column_order: {missing}")
        df = df.loc[:, list(column_order)]

    # --- PATCH: add total row at end (no other logic changes)
    if add_total_row:
        total_row = {c: "" for c in df.columns}

        for c in total_sum_cols:
            if c in df.columns:
                total_row[c] = df[c].sum()

        if total_label_col in df.columns:
            total_row[total_label_col] = total_label_text

        # keep SEQ/ลำดับ blank on total row (common report style)
        if seq_col in total_row:
            total_row[seq_col] = ""
        if seq_out_col in total_row:
            total_row[seq_out_col] = ""

        df = pd.concat([df, pd.DataFrame([total_row])], ignore_index=True)


    # export
    wb = export_vat_report(
        df,
        sheet_name=sheet_name,
        title_name=title_name,
        wb=wb,
        **export_kwargs
    )

    return df, wb

In [ ]:
def group_vat_by_date(df):
    df = df.copy()

    # ensure sorted so first/last BILLNO is correct
    df = df.sort_values(["วันที่", "เลขที่"])

    # define aggregation
    agg_dict = {
        "เลขที่": lambda x: f"{x.iloc[0]} - {x.iloc[-1]}",
        "มูลค่าสินค้า": "sum",
        "ภาษีมูลค่าเพิ่ม": "sum",
        "ยอดสุทธิ": "sum"
    }

    # all remaining columns → first
    for col in df.columns:
        if col not in ["วันที่", "เลขที่", "มูลค่าสินค้า", "ภาษีมูลค่าเพิ่ม", "ยอดสุทธิ"]:
            agg_dict[col] = "first"

    df_grouped = (
        df
        .groupby("วันที่", as_index=False)
        .agg(agg_dict)
    )

    # rebuild SEQ
    if "ลำดับ" in df_grouped.columns:
        df_grouped["ลำดับ"] = range(1, len(df_grouped) + 1)

    return df_grouped

In [ ]:
wb = None
wb_syp = None

In [ ]:
df_tar["ชื่อผู้ซื้อสินค้า"] = "คุณลูกค้าทั่วไป"
df_cntar["ชื่อผู้ซื้อสินค้า"] = "คุณลูกค้าทั่วไป"

df_3tar["ชื่อผู้ซื้อสินค้า"] = "คุณลูกค้าทั่วไป"
df_3cntar["ชื่อผู้ซื้อสินค้า"] = "คุณลูกค้าทั่วไป"

In [ ]:
column_order = [
    "ลำดับ",
    "วันที่",
    "เลขที่",
    "ชื่อผู้ซื้อสินค้า",
    "เลขผู้เสียภาษี",
    "มูลค่าสินค้า",
    "ภาษีมูลค่าเพิ่ม",
    "ยอดสุทธิ",
    "รายการสินค้า"
]

In [ ]:
rename_map = {
    "SEQ": "ลำดับ",
    "BILLDATE": "วันที่",
    "NEW_BILLNO": "เลขที่",
    "TAX_ID": "เลขผู้เสียภาษี",
    "BEFORE_VAT": "มูลค่าสินค้า",
    "VAT_AMOUNT": "ภาษีมูลค่าเพิ่ม",
    "TOTAL_AMOUNT": "ยอดสุทธิ",
    "DETAIL": "รายการสินค้า"
}

df_tar_report, wb = build_vat_report(
    df_tar,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="TAR",
    title_name="รายงานภาษีขาย (TAR)"
)

df_3tar_report, wb_syp = build_vat_report(
    df_3tar,
    rename_map,
    column_order,
    wb=wb_syp,
    sheet_name="3TAR",
    title_name="รายงานภาษีขาย (3TAR)",
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สาขาที่ 00003)",
    company_address="16/2 หมู่ 2 ต.ห้วยทับมอญ อ.เขาชะเมา จ.ระยอง 21110",
    tax_id="0215560000262",
)

In [ ]:
rename_map = {
    "SEQ": "ลำดับ",
    "BILLDATE": "วันที่",
    "NEG_BILLNO": "เลขที่",
    "TAX_ID": "เลขผู้เสียภาษี",
    "BEFORE_VAT": "มูลค่าสินค้า",
    "VAT_AMOUNT": "ภาษีมูลค่าเพิ่ม",
    "TOTAL_AMOUNT": "ยอดสุทธิ",
    "DETAIL": "รายการสินค้า"
}

df_cntar_report, wb = build_vat_report(
    df_cntar,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="CNTAR",
    title_name="รายงานภาษีขาย (CNTAR)"
)

df_3cntar_report, wb_syp = build_vat_report(
    df_3cntar,
    rename_map,
    column_order,
    wb=wb_syp,
    sheet_name="3CNTAR",
    title_name="รายงานภาษีขาย (3CNTAR)",
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สาขาที่ 00003)",
    company_address="16/2 หมู่ 2 ต.ห้วยทับมอญ อ.เขาชะเมา จ.ระยอง 21110",
    tax_id="0215560000262",
)

In [ ]:
rename_map = {
    "SEQ": "ลำดับ",
    "BILLDATE": "วันที่",
    "ACCTNAME": "ชื่อผู้ซื้อสินค้า",
    "BILLNO": "เลขที่",
    "REMARKS": "เลขผู้เสียภาษี",
    "BEFORETAX": "มูลค่าสินค้า",
    "TAX": "ภาษีมูลค่าเพิ่ม",
    "AFTERTAX": "ยอดสุทธิ",
    "DETAIL": "รายการสินค้า"
}


df_td_report, wb = build_vat_report(
    df_td,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="TD",
    title_name="รายงานภาษีขาย (TD)"
)

df_3td_report, wb_syp = build_vat_report(
    df_3td,
    rename_map,
    column_order,
    wb=wb_syp,
    sheet_name="3TD",
    title_name="รายงานภาษีขาย (3TD)",
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สาขาที่ 00003)",
    company_address="16/2 หมู่ 2 ต.ห้วยทับมอญ อ.เขาชะเมา จ.ระยอง 21110",
    tax_id="0215560000262",
)


In [ ]:
df_tr_report, wb = build_vat_report(
    df_tr,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="TR",
    title_name="รายงานภาษีขาย (TR)"
)

df_3tr_report, wb_syp = build_vat_report(
    df_3tr,
    rename_map,
    column_order,
    wb=wb_syp,
    sheet_name="3TR",
    title_name="รายงานภาษีขาย (3TR)",
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สาขาที่ 00003)",
    company_address="16/2 หมู่ 2 ต.ห้วยทับมอญ อ.เขาชะเมา จ.ระยอง 21110",
    tax_id="0215560000262",
)


df_tad_report, wb = build_vat_report(
    df_tad,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="TAD",
    title_name="รายงานภาษีขาย (TAD)"
)

In [ ]:
df_cn_report, wb = build_vat_report(
    df_cn,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="CN",
    title_name="รายงานภาษีขาย (CN)"
)

df_3cn_report, wb_syp = build_vat_report(
    df_3cn,
    rename_map,
    column_order,
    wb=wb_syp,
    sheet_name="3CN",
    title_name="รายงานภาษีขาย (3CN)",
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สาขาที่ 00003)",
    company_address="16/2 หมู่ 2 ต.ห้วยทับมอญ อ.เขาชะเมา จ.ระยอง 21110",
    tax_id="0215560000262",
)

df_cntad_report, wb = build_vat_report(
    df_cntad,
    rename_map,
    column_order,
    wb=wb,
    sheet_name="CNTAD",
    title_name="รายงานภาษีขาย (CNTAD)"
)

In [ ]:
def remove_total_row(df, label_col="เลขที่", label_text="รวม"):
    if label_col in df.columns:
        return df[df[label_col] != label_text].copy()
    return df.copy()

def strip_totals(dfs):
    return [remove_total_row(df) for df in dfs]

df_all = pd.concat(strip_totals([
    df_tar_report,
    df_cntar_report,
    df_td_report,
    df_tr_report,
    df_tad_report,
    df_cn_report,
    df_cntad_report,
]), ignore_index=True)

df_3all = pd.concat(strip_totals([
    df_3tar_report,
    df_3cntar_report,
    df_3td_report,
    df_3tr_report,
    df_3cn_report,
]), ignore_index=True)

/tmp/ipython-input-336/698166724.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_3all = pd.concat(strip_totals([


In [ ]:
df_all["วันที่"] = pd.to_datetime(df_all["วันที่"], errors="coerce")

df_all = df_all.sort_values(["วันที่", "เลขที่"]).reset_index(drop=True)

In [ ]:
df_3all["วันที่"] = pd.to_datetime(df_3all["วันที่"], errors="coerce")

df_3all = df_3all.sort_values(["วันที่", "เลขที่"]).reset_index(drop=True)

In [ ]:
df_all_report, wb = build_vat_report(
    df_all,
    wb=wb,
    sheet_name="รวม",
    title_name="รายงานภาษีขาย (รวม)",
    clean_text=False,   # ✅ important
    reset_seq=True
)

df_3all_report, wb_syp = build_vat_report(
    df_3all,
    wb=wb_syp,
    sheet_name="รวม",
    title_name="รายงานภาษีขาย (รวม)",
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สาขาที่ 00003)",
    company_address="16/2 หมู่ 2 ต.ห้วยทับมอญ อ.เขาชะเมา จ.ระยอง 21110",
    tax_id="0215560000262",
    clean_text=False,   # ✅ important
    reset_seq=True
)

In [ ]:
df_tar_daily = group_vat_by_date(remove_total_row(df_tar_report))

df_tar_daily_report, wb = build_vat_report(
    df_tar_daily,              # already cleaned/Thai columns is fine
    wb=wb,
    sheet_name="TAR (รายวัน)",
    title_name="รายงานภาษีขาย (TAR) สรุปรายวัน",
    clean_text=False   # ⭐ FIX
)

df_3tar_daily = group_vat_by_date(remove_total_row(df_3tar_report))

df_3tar_daily_report, wb_syp = build_vat_report(
    df_3tar_daily,              # already cleaned/Thai columns is fine
    wb=wb_syp,
    sheet_name="3TAR (รายวัน)",
    title_name="รายงานภาษีขาย (3TAR) สรุปรายวัน",
    company_name="บริษัท เกียรติชัยอะไหล่ยนต์ 2007 จำกัด (สาขาที่ 00003)",
    company_address="16/2 หมู่ 2 ต.ห้วยทับมอญ อ.เขาชะเมา จ.ระยอง 21110",
    tax_id="0215560000262",
    clean_text=False   # ⭐ FIX
)

In [ ]:
folder = os.path.join(
    BASE_FOLDER,
    "KCW-Data",
    "kcw_analytics",
    "04_outputs",
    "VAT_Sales_Report",
    f"vat_sales_report_{YEAR}_{MONTH:02d}_hq.xlsx"
)

# ⭐ create directory if missing
os.makedirs(os.path.dirname(folder), exist_ok=True)

wb.save(folder)

In [ ]:
folder = os.path.join(
    BASE_FOLDER,
    "KCW-Data",
    "kcw_analytics",
    "04_outputs",
    "VAT_Sales_Report",
    f"vat_sales_report_{YEAR}_{MONTH:02d}_syp.xlsx"
)

# ⭐ create directory if missing
os.makedirs(os.path.dirname(folder), exist_ok=True)

wb_syp.save(folder)